In [5]:
import numpy as np
from scipy.signal import convolve
from scipy.ndimage import gaussian_filter
from PIL import Image, ImageOps
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

# Sobel filters to find image gradients
sx, sy = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]]), np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

# A picture
img = 'cat.jpg'

def load_and_convert(loc):
    # Returns image pixel values stored in an ndarray #
    img = Image.open(loc)
    return np.asarray(img)#[:, :, ::4]

def greyscale(img, c_weights=[0.3, 0.59, 0.11]):
    # Returns ndarray with weight averaged RGB values #
    return np.einsum('abc,c->ab', img, c_weights)

def pad(img):
    # Turns nxm ndarray to (n+2)x(m+2) by mirroring boundary pixel values #
    img = np.c_[img, img[:, -1]]
    img = np.r_[img, [img[-1, :]]]
    img = np.c_[img[:, 0], img]
    return np.r_[[img[0, :]], img]
    
def pixel_coords(img):
    # Extracts coordinates of white pixels #
    coords = np.empty((img.shape[0]*img.shape[1],2), dtype=np.int64)
    c=0
    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            if img[y, x] == 255:
                coords[c,0]=x
                coords[c,1]=y
                c+=1
    return coords[:c,:]
    
def main(im, k=0.05, gsig=0.5, prec=10**6, rgb=True, show=True, analyse=True, opt=True, printit=False, K=100):
    # Main function #
    # Image.fromarray().show() just opens the images (takes a few seconds) 
    # and analysis part uses the EM algorithm to find the optimal amount and locations of corners
    # so set show and analyse to False to check the performance of the Harris algorithm. 
    img = load_and_convert(im)
    if rgb: img = greyscale(img)
    imgp = pad(img)
    gradx = convolve(imgp, sx, mode='valid') # Image gradients in x and y directions
    grady = convolve(imgp, sy, mode='valid')
    ix = gaussian_filter(gradx**2, gsig) # Applying Gaussian filters
    ixy = gaussian_filter(gradx*grady, gsig)
    iy = gaussian_filter(grady**2, gsig)
    det_m = ix * iy - ixy ** 2 # Determinant and trace of the structuree tensor
    tr_m = ix + iy
    r = det_m - k * tr_m ** 2
    edges, corners = (r<-prec) * 255, (r>prec) * 255
    if analyse: # Trying to find the amount and locations of corners
        pc = pixel_coords(edges)
        if opt: # Looking for optimal number of clusters
            k, BIC = 0, [] # Bayesian Information Criterion == BIC
            while k < K:
                k += 1
                gm = GaussianMixture(n_components=k).fit(pc) # EM algorithm to find the optimal amount of clusters and cluster centers
                BIC.append(np.log(gm.bic(pc)))
            plt.plot(range(k), BIC)
            plt.xlabel('Number of clusters')
            plt.ylabel('Log of Bayesian Information Criterion (BIC)')
            plt.show()
            nc = np.argmin(BIC) + 1 # Picking the amount of clusters that minimize BIC
        else: nc = K
        gm = GaussianMixture(n_components=nc).fit(pc)
        colors = np.random.choice(range(256), size=(nc, 3)) # Random colors for different clusters
        clustered_data = gm.predict(pc)
        if printit:
            print('Number of corners:' + str(len(gm.means_)))
            print('Corner coords:')
            for i in gm.means_:
                print(i.astype(int))
        clusters = np.zeros((corners.shape[0], corners.shape[1], 3), dtype='uint8')
        for c in range(len(pc)):
            clusters[pc[c][1]][pc[c][0]] = colors[clustered_data[c]]
    if show:
        Image.fromarray(img).show() # Image #1 - the image in grey scale
        Image.fromarray(edges).show() # Image #2 - the edges
        Image.fromarray(corners).show() # Image #3 - the corners
        Image.fromarray(np.array(clusters)).show() # Image #4 - the clustered corners
        Image.fromarray(np.array(edges+corners)).show()
    #IMG = ImageOps.invert(Image.fromarray(edges).convert('RGB'))
    #IMG.show()
    return clusters

if __name__ == "__main__":
    import time
    start_time = time.time()
    out = main(img, prec=10**6, show=True, gsig=0.5, analyse=True, opt=False, K=20) # Note: analysing the 2Mpx images takes a whiiiiile
    print("--- Time taken: %s s ---" % (time.time() - start_time))

--- Time taken: 34.739901304244995 s ---
